In [1]:
import configparser

config = configparser.ConfigParser()
config.read(r'C:\Users\GisselleMaira\test.ini')

import json
import os
import pandas as pd
import numpy as np
import datetime
from datetime import date
from datetime import datetime, timedelta
import shutil
import glob
import os.path
import os
import time
import urllib.request
import requests

In [2]:
dates = pd.read_excel (r'C:\Users\GisselleMaira\Desktop\SLT Docs\SeniorityReport.xlsx')

dates['user_name'] = dates['Preferred/First Name'] + ' ' + dates['Last Name'] 


dates.drop(['Last Name', 'Preferred/First Name', 'Tenure (In English)',  'Tenure (in Months)'], 1, inplace = True)

# dates.rename(columns = {'Hire Date':'Hire', 'Termination Date':'Term'})


In [3]:
today = date.today()
this_wk = today + timedelta(days=-today.weekday(), weeks=0)
start = today + timedelta(days=-today.weekday(), weeks=-21)

datelist = pd.date_range(start = start, periods = 21, freq='W-MON').tolist()
# datelist2 = pd.date_range(start = start, periods = 13, freq='W-SUN').tolist()
newdates = [d.strftime('%m-%d-%Y') for d in datelist]

active = datetime(2025, 1, 1)


In [4]:
newdates = pd.DataFrame(newdates) 
newdates['key'] = 1
dates['key'] = 1

dates2 = pd.merge(dates, newdates, on = 'key').drop('key', 1)

In [5]:
# dates2['end'] = dates2['Termination Date']
dates2.rename(columns={'Termination Date': 'end'}, inplace=True)
dates2["end"].fillna(active, inplace = True)

dates2['Active'] = 0
dates2['date']= pd.to_datetime(dates2[0])

# add 1 to Active if not term or dates before term
dates2.drop([0], 1, inplace=True)
dates2.loc[(dates2['Hire Date'] <= dates2['date']) & (dates2['end'] >= dates2['date']), 'Active'] = 1

# for i in range(0, len(datelist)):
#     dates[newdates[i]] = 0
    
#     dates.loc[(dates['Hire Date'] <= datelist[i]) &
#              (dates['end'] >= datelist[i]), newdates[i]] = 1

In [7]:
# pull API data for roles

acct = config['api']['account']
token = config['api']['token']

payload={}
headers = {
  'Harvest-Account-Id': acct,
  'Authorization': token
}

In [8]:
URL_users = f"https://api.harvestapp.com/v2/users?page=1&per_page=100"
all_users = []

while URL_users:
    print('Requesting', URL_users)
    
    response_users = requests.request("GET", URL_users, headers=headers, data=payload).text
    data_json_users  = json.loads(response_users)['users']
    all_users.extend(data_json_users)
    
    URL_users = json.loads(response_users)['links']['next']

Requesting https://api.harvestapp.com/v2/users?page=1&per_page=100
Requesting https://api.harvestapp.com/v2/users?page=2&per_page=100&ref=next


In [9]:
users = pd.json_normalize(all_users,max_level=1)[['first_name','last_name', 'email','roles']]
users2 = users.explode('roles')
users2['user_name'] = users2['first_name'] + ' ' + users2['last_name']
users2.drop(['first_name', 'last_name', 'email'], 1, inplace =True)
users2.reset_index(drop=True, inplace=True)

In [10]:
users2.drop(users2.loc[users2['user_name'].isin(['Parker Barouch', 'Kevin Baquero']) & 
                       (users2['roles'] == 'Flying Phase')].index, inplace = True)

# users2.loc[users2['user_name'].isin(['Parker Barouch', 'Kevin Baquero']) ]

In [11]:
role1 = users2[users2['roles'].isin(['Advisor', 'Consultant'])]

role2 = users2[users2['roles'].isin(['CCOM', 'Risk', 'Data and Analytics','Flying Phase'])]

user_roles = pd.merge(role1, role2, on='user_name', how='left')

user_roles = user_roles.rename(columns = {'roles_x': 'role', 'roles_y': 'BusUnit'})

In [15]:
empcounts = pd.merge(dates2, user_roles, on='user_name', how='left')

empcounts2 = empcounts[['user_name', 'role', 'BusUnit', 'Hire Date', 'end', 'date', 'Active']] 

empcounts2.to_excel( r'C:\Users\GisselleMaira\Desktop\SLT Docs\DashboardData\empcount.xlsx', index = False)

In [16]:
empcounts2.head()

,user_name,role,BusUnit,Hire Date,end,date,Active
0,Shawn Sweeney,NaN,NaN,2016-12-28,2025-01-01,2021-09-13,1
1,Shawn Sweeney,NaN,NaN,2016-12-28,2025-01-01,2021-09-20,1
2,Shawn Sweeney,NaN,NaN,2016-12-28,2025-01-01,2021-09-27,1
3,Shawn Sweeney,NaN,NaN,2016-12-28,2025-01-01,2021-10-04,1
4,Shawn Sweeney,NaN,NaN,2016-12-28,2025-01-01,2021-10-11,1
